In [ ]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation, Conv2D, Flatten, Dense,Dropout
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adadelta, Adagrad, Adam, Adamax, RMSprop, Nadam
from PIL import Image
import numpy as np
import glob
import matplotlib.pyplot as plt
import time
import os
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

In [ ]:
os.chdir('..')
!ls

bin					   etc	  opt	 sys
boot					   home   proc	 tensorflow-2.1.0
content					   lib	  root	 tmp
datalab					   lib32  run	 tools
dev					   lib64  sbin	 usr
dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl  media  srv	 var
dlib-19.18.0-cp36-cp36m-linux_x86_64.whl   mnt	  swift


In [ ]:
#フォルダ一覧の読み込み
folder = os.listdir("dot_rotated")
folder.sort()
# folder.pop(0)
width,height = 58,52
dense_size  = len(folder)
#画像データとラベル
X = []
Y = []

In [ ]:
for index, name in enumerate(folder[:50]):
    dir = "./dot_rotated/" + name
    files = glob.glob(dir + "/*.png")
    print(name)
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        data = np.asarray(image)
        X.append(data)
        Y.append(index)

001
002
003
003m1
004
005
006
006m1
006m2
007
008
009
009m1
010
011
012
013
014
015
015m1
016
017
018
018m1
019
019a
020
020a
021
022
023
024
025
026
026a
027
027a
028
028a
029
030
031
032
033
034
035
036
037
037a
038


In [ ]:
X = np.array(X)
Y = np.array(Y)
X = X.astype('float32')
X = X / 255.0

In [ ]:
Y = np_utils.to_categorical(Y, dense_size)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10)

In [ ]:
#CNNの層
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(dense_size))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 52, 58, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 52, 58, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 50, 56, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 50, 56, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 28, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 25, 28, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 25, 28, 64)       

In [ ]:
#CNNの学習器の作成
optimizers ="Adadelta"
results = {}
fpath = './tensorlog/weights.{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5'
cp_cb = ModelCheckpoint(filepath = fpath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
tb_cb = TensorBoard(log_dir="./tensorlog", histogram_freq=1)
epochs = 200
model.compile(loss='categorical_crossentropy', optimizer=optimizers, metrics=['accuracy'])
results= model.fit(X_train, y_train, validation_split=0.2, epochs=epochs, callbacks=[cp_cb, es_cb, tb_cb])

Train on 1116 samples, validate on 279 samples



Epoch 1/200
1116/1116 [==============================] - 21s 19ms/step - loss: 0.0010 - acc: 0.9991 - val_loss: 2.2069e-07 - val_acc: 1.0000

Epoch 00001: val_loss improved from inf to 0.00000, saving model to ./tensorlog/weights.01-0.00-0.00.hdf5

Epoch 2/200
1116/1116 [==============================] - 19s 17ms/step - loss: 0.0027 - acc: 0.9991 - val_loss: 1.7433e-07 - val_acc: 1.0000

Epoch 00002: val_loss improved from 0.00000 to 0.00000, saving model to ./tensorlog/weights.02-0.00-0.00.hdf5
Epoch 3/200
1116/1116 [==============================] - 19s 17ms/step - loss: 0.0018 - acc: 0.9982 - val_loss: 2.7038e-06 - val_acc: 1.0000

Epoch 00003: val_loss did not improve from 0.00000
Epoch 4/200
1116/1116 [==============================] - 19s 17ms/step - loss: 8.4756e-04 - acc: 1.0000 - val_loss: 1.4057e-07 - val_acc: 1.0000

Epoch 00004: val_loss improved from 0.00000 to 0.00000, saving model to ./tensorlog/weights.04-0.00-0.00.hdf5


In [ ]:
# 推定
score = model.evaluate(X_test, y_test, batch_size=32, verbose=0)
print('validation loss:{0[0]}\nvalidation accuracy:{0[1]}'.format(score))

validation loss:0.10398786529421188
validation accuracy:0.9935483909422351


In [ ]:
model_json_str = model.to_json()
open('pokeAI_mlp_model.json', 'w').write(model_json_str)
model.save_weights('pokeAI_mlp_weights.h5');

In [ ]:
import pickle
with open('PokeAI.pickle', mode='wb') as fp:
  pickle.dump(model, fp)

In [ ]:
!pip install -U coremltools

     |████████████████████████████████| 3.4MB 2.1MB/s 


In [ ]:
labels = open("./labels.txt","rb")
list_row = pickle.load(f)

EOFError: ignored

In [ ]:
import coremltools
coremltools.converters.keras.convert('pokeAI_mlp_weights.h5',
   input_names = 'image',
   output_names = ['classLabelProbs', 'classLabel'],
   image_input_names = 'image',
   class_labels='labels.txt').save('pokeAI_mlp_weights.mlmodel')

In [ ]:
labels=[
"001",
"002",
"003",
"003m1",
"004",
"005",
"006",
"006m1",
"006m2",
"007",
"008",
"009",
"009m1",
"010",
"011",
"012",
"013",
"014",
"015",
"015m1",
"016",
"017",
"018",
"018m1",
"019",
"019a",
"020",
"020a",
"021",
"022",
"023",
"024",
"025",
"026",
"026a",
"027",
"027a",
"028",
"028a",
"029",
"030",
"031",
"032",
"033",
"034",
"035",
"036",
"037",
"037a",
"038"]

In [ ]:
coreml_model = coremltools.converters.keras.convert(model, input_names='image', image_input_names='image', 
                                                    class_labels=labels) 


0 : conv2d_5_input, <keras.engine.input_layer.InputLayer object at 0x7f302fb2b160>
1 : conv2d_5, <keras.layers.convolutional.Conv2D object at 0x7f302fd170f0>
2 : activation_7, <keras.layers.core.Activation object at 0x7f302fb29c18>
3 : conv2d_6, <keras.layers.convolutional.Conv2D object at 0x7f302fb29940>
4 : activation_8, <keras.layers.core.Activation object at 0x7f302cc00b00>
5 : max_pooling2d_3, <keras.layers.pooling.MaxPooling2D object at 0x7f302fd17c50>
6 : conv2d_7, <keras.layers.convolutional.Conv2D object at 0x7f302fb2ac18>
7 : activation_9, <keras.layers.core.Activation object at 0x7f302fb295f8>
8 : conv2d_8, <keras.layers.convolutional.Conv2D object at 0x7f302cba4f28>
9 : activation_10, <keras.layers.core.Activation object at 0x7f302cba4898>
10 : max_pooling2d_4, <keras.layers.pooling.MaxPooling2D object at 0x7f302cbbc1d0>
11 : flatten_2, <keras.layers.core.Flatten object at 0x7f302cbc36d8>
12 : dense_3, <keras.layers.core.Dense object at 0x7f302cbc3198>
13 : activation_11, <

In [ ]:
coreml_model.save('PokeAI.mlmodel') 